In [4]:
import numpy as np
import matplotlib.pyplot as plt


mhatree_to_kcal = 0.6275094740631

train_set, eval__set = [], []
train_set.append([185.89158003098848, 445.34157216716665])
eval__set.append([178.75854150564686, 662.4498123299053])

train_set.append([68.57623712544817, 149.1687403205493])
eval__set.append([108.49399374936273, 315.3520668758974])

train_set.append([63.83055009148214, 130.2752650182697])
eval__set.append([84.20118550610661, 264.7708749861897])

# train_set.append([146.696, 376.323, 1159])
# eval__set.append([169.518, 632.872, 244])

train_set, eval__set = np.array(train_set), np.array(eval__set)
train_set[:, 0] = mhatree_to_kcal * train_set[:, 0]
eval__set[:, 0] = mhatree_to_kcal * eval__set[:, 0]

print(np.array_str(train_set[-1, :], precision=2, suppress_small=True))
print(np.array_str(eval__set[-1, :], precision=2, suppress_small=True))

[ 43.03 149.17]
[ 68.08 315.35]


In [ ]:
plt.plot(train_set[:, 0], "r")
plt.plot(eval__set[:, 0], "b")
plt.plot(train_set[:, 1], "r--")
plt.plot(eval__set[:, 1], "b--")
plt.show()